In [ ]:
# 🧰 1) Setup: clone repo, set paths, install deps
!rm -rf /kaggle/working/MedVLMBenchPhase1
!git clone https://github.com/dimplek0424/MedVLMBenchPhase1.git
%cd /kaggle/working/MedVLMBenchPhase1

# Core deps
!pip install --quiet "torch>=2.1.0" "torchvision" "timm" "wget" "pyyaml" "pandas"
# BioMedCLIP via open-clip
!pip install --quiet "open-clip-torch>=2.24.0"
# (Optional) If you need to pull weights from HF hub, uncomment:
# import os; os.environ["BIOMEDCLIP_HF_REPO"]="microsoft/BiomedCLIP-PubMedBERT-base-uncased-ViT-B-16"

import os, sys, glob
os.environ["DATA_DIR"]   = "/kaggle/input/chest-xrays-indiana-university"
os.environ["OUTPUT_DIR"] = "/kaggle/working/outputs"

print("python:", sys.executable)
print("REPO_DIR:", os.getcwd())
print("DATA_DIR:", os.getenv("DATA_DIR"))
print("OUTPUT_DIR:", os.getenv("OUTPUT_DIR"))
!ls -alh configs


In [ ]:
# ✅ 2) Sanity: check imports and file presence
import pathlib, yaml, torch
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())

cfg_path = pathlib.Path("configs/dataset_iu_v03_full.yaml")
print("cfg exists:", cfg_path.exists())
print("images dir exists:",
      pathlib.Path("configs").exists())

p = pathlib.Path("scripts/projection_biomedclip.py")
print("biomedclip runner exists:", p.exists(), "size:", p.stat().st_size if p.exists() else "NA")


In [ ]:
# 🚀 3) Run BioMedCLIP projection
import os, sys, subprocess
os.makedirs("/kaggle/working/outputs", exist_ok=True)
os.makedirs("results/projection", exist_ok=True)

cfg  = "configs/dataset_iu_v03_full.yaml"
task = "configs/task_projection_v01.yaml"
out  = "results/projection/iu_v03_full_biomedclip.csv"

cmd = [sys.executable, "scripts/projection_biomedclip.py",
       "--config", cfg, "--task", task, "--out", out]
print("Running:", " ".join(cmd))
subprocess.check_call(cmd)
print("\n✅ Done. See", out)


In [ ]:
# 🔍 4) Quick smoke check
import os, pandas as pd
csv_path = "results/projection/iu_v03_full_biomedclip.csv"
assert os.path.exists(csv_path), "Output CSV not found."
df = pd.read_csv(csv_path)
print(df.head(3))
print("\nRows:", len(df))


In [ ]:
# 📊 5) Evaluation (confusion matrix, etc.)
PRED_CSV = "results/projection/iu_v03_full_biomedclip.csv"
!python scripts/evaluate_views.py \
  --csv {PRED_CSV} \
  --col_image image \
  --col_p1 p_frontal \
  --col_p2 p_lateral \
  --col_pred pred \
  --outdir results/eval_biomedclip

!ls -lh results/eval_biomedclip
!zip -r /kaggle/working/eval_biomedclip_artifacts.zip results/eval_biomedclip
